<a href="https://colab.research.google.com/github/FrancescoI/RecoUtils/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import torch
import pandas as pd
import numpy as np
from torch.autograd import Variable
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder

In [51]:
def gpu(tensor, gpu=False):

    if gpu:
        return tensor.cuda()
    else:
        return tensor


def cpu(tensor):

    if tensor.is_cuda:
        return tensor.cpu()
    else:
        return tensor

In [52]:
### Loss

In [53]:
def hinge_loss(positive, negative):
    
    loss = torch.clamp(negative - positive + 1.0, 0.0)

    return loss.mean()

In [54]:
### Utilities

In [55]:
def get_negative_batch(batch, n_items, item_to_metadata_dict):
    
    neg_batch = None

    neg_item_id = np.random.randint(0, n_items-1, len(batch))
    
    if item_to_metadata_dict:
        neg_metadata_id = [item_to_metadata_dict[item] for item in neg_item_id]    
    else:
        neg_metadata_id = None
    
    neg_batch = pd.concat([neg_batch, pd.DataFrame({'user_id': batch['user_id'],
                                                    'item_id': neg_item_id, 
                                                    'metadata': neg_metadata_id})])
            
    return neg_batch

In [56]:
class Dataset():
    
    def __init__(self, brand):
        
        self.brand = brand
    
    def _get_interactions(self):

        bucket_uri = f'/content/drive/My Drive/'

        if self.brand == 'missoni':
        
          dataset = pd.read_csv(bucket_uri + f'{self.brand}.csv')


        elif self.brand == 'ton':
          
          clickstream = pd.read_csv(bucket_uri + f'{self.brand}.csv')
          metadata = pd.read_csv(bucket_uri + 'anagrafica_ton.csv')

          clickstream = (clickstream\
                         .groupby(['user_ids', 'product_code'])['brand'].count() 
                         ).reset_index()
          
          clickstream.columns = ['hashedEmail', 'product', 'actions']

          dataset = pd.merge(clickstream, metadata, left_on='product', right_on='pty_pim_variant')
          
          dataset = dataset[['hashedEmail', 'product', 'macro', 'saleline', 'actions']]
          dataset.columns = ['hashedEmail', 'product', 'macro', 'saleLine', 'actions']

          dataset['gender'] = 'W'

        return dataset 
    
    
    def _encondig_label(self, dataset, input_col, output_col):
        
        encoder = LabelEncoder()
        dataset[output_col] = encoder.fit(dataset[input_col]).transform(dataset[input_col])
        
        return dataset, encoder
    
    
    def fit(self, metadata=None, seasons=None):
        
        dataset = self._get_interactions()
        self.metadata = metadata
        
        if seasons:
            dataset = dataset[dataset['season'].isin(seasons)]
        
        ### Label Encoding
        dataset, _ = self._encondig_label(dataset, input_col='hashedEmail', output_col='user_id')
        dataset, _ = self._encondig_label(dataset, input_col='product', output_col='item_id')
        
        if metadata is not None:
            output_list_name = []
            
            for meta in metadata:
                output_name = meta + '_id'
                dataset, _ = self._encondig_label(dataset, input_col=meta, output_col=output_name)
                output_list_name.append(output_name)                
            
            dataset['metadata'] = dataset[output_list_name].values.tolist()
            self.metadata_id = output_list_name
            
        self.dataset = dataset
        
    def get_item_metadata_dict(self):
        
        if self.metadata is not None:
        
            return self.dataset.set_index('item_id')['metadata'].to_dict()
        
        else:
            
            return None

In [57]:
### Evaluate

In [58]:
def auc_score(positive, negative):
    
    total_auc = []
    
    positive = positive.cpu().detach().numpy()
    negative = negative.cpu().detach().numpy()

    batch_auc = (positive > negative).sum() / len(positive)
    total_auc.append(batch_auc)
        
    return np.mean(total_auc)

In [59]:
def evaluate(model):
    
    ### TRAIN    
    negative_train = get_negative_batch(batch=model.train, n_items=model.dataset.dataset['item_id'].max(), item_to_metadata_dict=None)
    negative_train.columns = ['user_id', 'item_id_neg', 'metadata_neg']
    
    merged_train = pd.concat([model.train, negative_train], axis=1)
    
    train_auc = []
    
    for row in merged_train.itertuples():
        
        score = np.sum(model.pred[row.user_id, row.item_id] > model.pred[row.user_id, row.item_id_neg])
        train_auc.append(score)
    
    ### TEST
    negative_test = get_negative_batch(batch=model.test, n_items=model.dataset.dataset['item_id'].max(), item_to_metadata_dict=None)
    negative_test.columns = ['user_id', 'item_id_neg', 'metadata_neg']
    merged_test = pd.concat([model.test, negative_test], axis=1)
    
    test_auc = []
    
    for row in merged_test.itertuples():
        
        score = np.sum(model.pred[row.user_id, row.item_id] > model.pred[row.user_id, row.item_id_neg])
        test_auc.append(score)
    
    print(f'Train AUC: {np.sum(train_auc) / merged_train.shape[0]}')
    print(f'Test AUC: {np.sum(test_auc) / merged_test.shape[0]}')

In [60]:
def precision_recall_k(model, k):
    
    ### TRAIN
    values = np.ones(model.train.shape[0])
    users = model.train.loc[:, 'user_id']
    items = model.train.loc[:, 'item_id']
    
    sparse_matrix = sp.csr_matrix((values, (users, items)))
    
    matrix = sparse_matrix.toarray()
    total_precision = []
    total_recall = []
    
    for index, row in enumerate(matrix):
        
        truth = np.nonzero(row)[0]
        if len(truth) > 0:
            prediction = np.argsort(-model.pred[index, :])[:k]

            n_matching = len(set(truth) & set(prediction))
            precision = n_matching / k
            recall = n_matching / len(truth)

            total_precision.append(precision)
            total_recall.append(recall)
        
    print(f'Train Precision@{k}: {np.mean(total_precision)} \nTrain Recall@{k}: {np.mean(total_recall)} \nTrain Shape: {sparse_matrix.getnnz()}')
        
    
    ### TEST
    values = np.ones(model.test.shape[0])
    users = model.test.loc[:, 'user_id']
    items = model.test.loc[:, 'item_id']
    
    sparse_matrix = sp.csr_matrix((values, (users, items)))
    
    matrix = sparse_matrix.toarray()
    total_precision = []
    total_recall = []
    
    for index, row in enumerate(matrix):
        
        truth = np.nonzero(row)[0]
        if len(truth) > 0:
            prediction = np.argsort(-model.pred[index, :])[:k]

            n_matching = len(set(truth) & set(prediction))
            precision = n_matching / k
            recall = n_matching / len(truth)

            total_precision.append(precision)
            total_recall.append(recall)
        
    print(f'\nTest Precision@{k}: {np.mean(total_precision)} \nTest Recall@{k}: {np.mean(total_recall)} \nTest Shape: {sparse_matrix.getnnz()}')

In [61]:
### Model

In [62]:
class ScaledEmbedding(torch.nn.Embedding):
    """
    Embedding layer that initialises its values
    to using a normal variable scaled by the inverse
    of the embedding dimension.
    """

    def reset_parameters(self):
        """
        Initialize parameters.
        """

        self.weight.data.normal_(0, 1.0 / self.embedding_dim)
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)

In [63]:
class ZeroEmbedding(torch.nn.Embedding):
    """
    Embedding layer that initialises its values
    to using a normal variable scaled by the inverse
    of the embedding dimension.
    Used for biases.
    """

    def reset_parameters(self):
        """
        Initialize parameters.
        """

        self.weight.data.zero_()
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)

In [64]:
class CFTemplate(torch.nn.Module):
    
    def __init__(self, dataset, n_factors, use_cuda=False):
        super().__init__()
             
        self.dataset = dataset
        self.n_users = dataset.dataset['user_id'].max() + 1
        self.n_items = dataset.dataset['item_id'].max() + 1
        self.dictionary = dataset.get_item_metadata_dict()
        
        self.n_factors = n_factors
        
        self.use_cuda = use_cuda

        self.user = gpu(ScaledEmbedding(self.n_users, self.n_factors), self.use_cuda)
        self.item = gpu(ScaledEmbedding(self.n_items, self.n_factors), self.use_cuda)
        
        self.user_bias = gpu(ZeroEmbedding(self.n_users, 1), self.use_cuda)
        self.item_bias = gpu(ZeroEmbedding(self.n_items, 1), self.use_cuda)
        
       
    def forward(self, net, batch, batch_size=1):
        
        neg_batch = get_negative_batch(batch, self.n_items, self.dictionary)
        
        positive = gpu(self.net(batch, batch_size), self.use_cuda)
        negative = gpu(self.net(neg_batch, batch_size), self.use_cuda)
        
        return positive, negative
    
    
    def backward(self, positive, negative, optimizer):
                
        optimizer.zero_grad()
                
        loss_value = hinge_loss(positive, negative)                

        loss_value.backward()
        
        optimizer.step()
        
        return loss_value.item()
    
    
    def fit(self, optimizer, batch_size=1024, epochs=10, split_train_test=False, verbose=False):
        
        if split_train_test:
            
            print('|== Splitting Train/Test ==|')
            
            data = self.dataset.dataset.iloc[np.random.permutation(len(self.dataset.dataset))]
            train = data.iloc[:int(len(data) * 0.9)]
            test = data.iloc[int(len(data) * 0.9):]
            
            print(f'Shape Train: {len(train)} \nShape Test: {len(test)}')
            
        else:
            
            train = self.dataset.dataset
        
        
        self.total_train_auc = []
        self.total_test_auc = []
        self.total_loss = []
        
        for epoch in range(epochs):

            print(f'Epoch: {epoch+1}')
            
            epoch_loss = []

            for first in range(0, len(train), batch_size):
                
                batch = train.iloc[first:first+batch_size, :]     
                                
                net = self.net(batch, len(batch))
                
                positive, negative = self.forward(net, batch, len(batch))
                
                loss_value = self.backward(positive, negative, optimizer)
                
            epoch_loss.append(loss_value)
            self.total_loss.append(epoch_loss)
            
            if verbose:
                ### AUC Calc.

                train_sample = train.sample(n=20_000)

                net_train = self.net(train_sample, len(train_sample))
                positive_train, negative_train = self.forward(net_train, train_sample, len(train_sample))           
                train_auc = auc_score(positive_train, negative_train)
                self.total_train_auc.append(train_auc)

                test_sample = test.sample(n=20_000) 
                
                net_test = self.net(test_sample, len(test_sample))
                positive_test, negative_test = self.forward(net_test, test_sample, len(test_sample))
                test_auc = auc_score(positive_test, negative_test)
                self.total_test_auc.append(test_auc)
                
                print(f'Epoch {epoch+1}, \n== Loss: {sum(epoch_loss)} \n== Train AUC: {train_auc} \n== Test AUC: {test_auc}')
            
    def history(self):
        
        return {'train_loss': self.total_loss,
                'train_auc': self.total_train_auc,
                'test_auc': self.total_test_auc}
    
    def get_item_representation(self):
        
        return self.item.cpu().detach().numpy()

In [65]:
class LightFM(CFTemplate):
    
    def __init__(self, dataset, n_factors, use_metadata=True, use_cuda=False):
        super().__init__(dataset, n_factors, use_cuda)
        
        self.use_metadata = use_metadata
        self.use_cuda = use_cuda
        
        if use_metadata:
            self.n_metadata = self._get_n_metadata(dataset)
            self.metadata = gpu(ScaledEmbedding(self.n_metadata, n_factors), self.use_cuda)
    
    
    def _get_n_metadata(self, dataset):
        
        n_metadata = 0
        
        for col in dataset.metadata_id:
            n_metadata += dataset.dataset[col].max() + 1
        
        return n_metadata
    
    
    def net(self, batch, batch_size):
        
        """
        Forward method that express the model as the dot product of user and item embeddings, plus the biases. 
        Item Embeddings itself is the sum of the embeddings of the item ID and its metadata
        """
        
        user = Variable(gpu(torch.LongTensor(batch['user_id'].values), self.use_cuda))
        item = Variable(gpu(torch.LongTensor(batch['item_id'].values), self.use_cuda))
        
        if self.use_metadata:
            metadata = Variable(gpu(torch.LongTensor(list(batch['metadata'])), self.use_cuda))
            metadata = self.metadata(metadata)

        user_bias = self.user_bias(user)
        item_bias = self.item_bias(item)
        
        user = self.user(user)
        item = self.item(item)
        
        if self.use_metadata:
        
            ### Reshaping in order to match metadata tensor
            item = item.reshape(batch_size, 1, self.n_factors)        
            item_metadata = torch.cat([item, metadata], axis=1)

            ### sum of latent dimensions
            item = item_metadata.sum(1)
        
        net = (user * item).sum(1).view(-1,1) + user_bias + item_bias
        
        return net
    
    def get_item_representation(self):
        
        if self.use_metadata:
            
            data = (self.dataset
                    .dataset[['item_id'] + self.dataset.metadata_id]
                    .drop_duplicates())
            
            mapping = pd.get_dummies(data, columns=[*self.dataset.metadata_id]).values[:, 1:]
            identity = np.identity(self.dataset.dataset['item_id'].max() + 1)
            binary = np.hstack([identity, mapping])
            
            metadata_representation = np.vstack([self.item.weight.detach().numpy(), self.metadata.weight.detach().numpy()])
            
            return np.dot(binary, metadata_representation), binary, metadata_representation
        
        else:
            return self.item.weight.cpu().detach().numpy()
        
        
    def predict(self, user_idx):
        
        """
        It takes a user vector representation (based on user_idx arg) and it takes the dot product with
        the item representation
        """
        
        item_repr, _, _ = self.get_item_representation()
        user_repr = self.user.weight.detach().numpy()
        
        item_bias = self.item_bias.weight.detach().numpy()
        user_bias = self.user_bias[torch.tensor([user_idx])].detach().numpy()
        
        return np.dot(user_pred[user_idx, :], item_repr) + item_bias + user_bias

In [66]:
class MLP(CFTemplate):
    
    def __init__(self, dataset, n_factors, use_metadata=True, use_cuda=False):
        super().__init__(dataset, n_factors, use_cuda)
        
        self.use_metadata = use_metadata
        self.use_cuda = use_cuda

        if use_metadata:
            self.n_metadata = self._get_n_metadata(dataset)
            self.n_metadata_type = self._get_n_metadata_type(dataset)
            self.metadata = gpu(ScaledEmbedding(self.n_metadata, n_factors), self.use_cuda)
            
        else:
            self.n_metadata_type = 0
        
        self.linear_1 = gpu(torch.nn.Linear(n_factors*(2+self.n_metadata_type), int(self.n_factors/2)), self.use_cuda)
        self.linear_2 = gpu(torch.nn.Linear(int(self.n_factors/2), int(self.n_factors/4)), self.use_cuda)
        self.linear_3 = gpu(torch.nn.Linear(int(self.n_factors/4), 1), self.use_cuda)
        
    def _get_n_metadata(self, dataset):
        
        n_metadata = 0
        
        for col in dataset.metadata_id:
            n_metadata += dataset.dataset[col].max() + 1
        
        return n_metadata
    
    def _get_n_metadata_type(self, dataset):
        
        return len(dataset.metadata)
    
    
    def mlp(self, dataset, batch_size=1):
        
        """
        """
        user = gpu(torch.from_numpy(dataset['user_id'].values), self.use_cuda)
        item = gpu(torch.from_numpy(dataset['item_id'].values), self.use_cuda)
        
        if self.use_metadata:
            metadata = Variable(gpu(torch.LongTensor(list(dataset['metadata'])), self.use_cuda))
            metadata = self.metadata(metadata).reshape(batch_size, self.n_factors*self.n_metadata_type)
            
        user = self.user(user)
        item = self.item(item)
        
        if self.use_metadata:
            cat = torch.cat([user, item, metadata], axis=1).reshape(batch_size, (2+self.n_metadata_type)*self.n_factors)
        else:
            cat = torch.cat([user, item], axis=1).reshape(batch_size, 2*self.n_factors)
                
        net = self.linear_1(cat)
        net = torch.nn.functional.relu(net)
        
        net = self.linear_2(net)
        net = torch.nn.functional.relu(net)
        
        net = self.linear_3(net)
        
        return net
    
    def net(self, dataset, batch_size=1):
        
        """
        """
        
        net = gpu(self.mlp(dataset, batch_size), self.use_cuda)
                
        return net
    
    def get_item_representation(self):
        
        if self.use_metadata:
            
            data = (self.dataset
                    .dataset[['item_id'] + self.dataset.metadata_id]
                    .drop_duplicates())
            
            mapping = pd.get_dummies(data, columns=[*self.dataset.metadata_id]).values[:, 1:]
            identity = np.identity(self.dataset.dataset['item_id'].max() + 1)
            binary = np.hstack([identity, mapping])
            
            metadata_representation = np.vstack([self.item.weight.detach().numpy(), self.metadata.weight.detach().numpy()])
            
            return np.dot(binary, metadata_representation), binary, metadata_representation
        
        else:
            return self.item.weight.cpu().detach().numpy()

In [67]:
class NeuCF(CFTemplate):
    
    def __init__(self, dataset, n_factors, use_metadata=True):
        super().__init__(dataset, n_factors)
        
        self.use_metadata = use_metadata
        
        if use_metadata:
            self.n_metadata = self._get_n_metadata(dataset)
            self.n_metadata_type = self._get_n_metadata_type(dataset)
            self.metadata_gmf = ScaledEmbedding(self.n_metadata, n_factors)
            self.metadata_mlp = ScaledEmbedding(self.n_metadata, n_factors)
            
        else:
            self.n_metadata_type = 0
    
        self.user_gmf = ScaledEmbedding(self.n_users, self.n_factors)
        self.user_mlp = ScaledEmbedding(self.n_users, self.n_factors)
        
        self.item_gmf = ScaledEmbedding(self.n_items, self.n_factors)
        self.item_mlp = ScaledEmbedding(self.n_items, self.n_factors)
        
        self.linear_1 = torch.nn.Linear(n_factors*(2+self.n_metadata_type), self.n_factors*4)
        self.linear_2 = torch.nn.Linear(self.n_factors*4, self.n_factors*2)
        self.linear_3 = torch.nn.Linear(self.n_factors*2, self.n_factors)
        self.linear_4 = torch.nn.Linear(self.n_factors*2, 1)
        
        self.weights = torch.nn.Parameter(torch.rand(2), requires_grad=True)
        
    def _get_n_metadata(self, dataset):
        
        n_metadata = 0
        
        for col in dataset.metadata_id:
            n_metadata += dataset.dataset[col].max() + 1
        
        return n_metadata
    
    def _get_n_metadata_type(self, dataset):
        
        return len(dataset.metadata)
    
        
    def gmf(self, dataset, batch_size=1):
        
        """
        """
        
        user = Variable(torch.LongTensor(dataset['user_id'].values))
        item = Variable(torch.LongTensor(dataset['item_id'].values))
        
        if self.use_metadata:
            metadata = Variable(torch.LongTensor(list(dataset['metadata'])))
            metadata = self.metadata_gmf(metadata)
            
        user = self.user_gmf(user)
        item = self.item_gmf(item)
        
        if self.use_metadata:
            item = item.reshape(batch_size, 1, self.n_factors)        
            item_metadata = torch.cat([item, metadata], axis=1)

            ### sum of latent dimensions
            item = item_metadata.sum(1)
        
        #net = (user * item).sum(1).view(-1,1) 
        net = (user * item)
        
        return net
    
    def mlp(self, dataset, batch_size=1):
        
        """
        """
        user = Variable(torch.LongTensor(dataset['user_id'].values))
        item = Variable(torch.LongTensor(dataset['item_id'].values))
        
        if self.use_metadata:
            metadata = Variable(torch.LongTensor(list(dataset['metadata'])))
            metadata = self.metadata_mlp(metadata).reshape(batch_size, self.n_factors*self.n_metadata_type)
            
        user = self.user_mlp(user)
        item = self.item_mlp(item)
        
        if self.use_metadata:
            cat = torch.cat([user, item, metadata], axis=1).reshape(batch_size, (2+self.n_metadata_type)*self.n_factors)
        else:
            cat = torch.cat([user, item], axis=1).reshape(batch_size, 2*self.n_factors)
                
        net = self.linear_1(cat)
        net = torch.nn.functional.relu(net)
        
        net = self.linear_2(net)
        net = torch.nn.functional.relu(net)
        
        net = self.linear_3(net)
        
        return net
    
    def net(self, dataset, batch_size=1):
        
        """
        """
        user = Variable(torch.LongTensor(dataset['user_id'].values))
        item = Variable(torch.LongTensor(dataset['item_id'].values))
        
        gmf = self.gmf(dataset, batch_size)
        mlp = self.mlp(dataset, batch_size)
        
        net = torch.cat([gmf, mlp], axis=1)
        
        net = self.linear_4(net)
#        net = torch.nn.functional.sigmoid(net)
#         net = (self.weights * net).sum(1)
                
        return net

In [68]:
class EASE():
    
    def __init__(self, dataset, split_train_test=True):
        
        self.dataset = dataset
        self.item_dict = dataset.dataset.set_index('item_id')['product'].to_dict()
        self.split_train_test = split_train_test
        self._split_train_test()
        
    
    def _split_train_test(self):
        
        if self.split_train_test:
        
            print('|== Splitting Train/Test ==|')
            
            data = self.dataset.dataset.iloc[np.random.permutation(len(self.dataset.dataset))]
            self.train = data.iloc[:int(len(data) * 0.9)]
            self.test = data.iloc[int(len(data) * 0.9):]
            
            print(f'Shape Train: {len(self.train)} \nShape Test: {len(self.test)}')
            
        else:
            
            self.train = self.dataset.dataset
            
    
    def fit(self, lambda_: float = 0.5, implicit=True):
        
        if implicit:
            values = np.ones(self.train.shape[0])
        else:
            values = self.train.loc[:, 'action']
        
        users = self.train.loc[:, 'user_id']
        items = self.train.loc[:, 'item_id']
        
        matrix = sp.csr_matrix((values, (users, items)))
        self.matrix = matrix
        
        ### Weight Bij are
        ### 0s if i=j (diagonal)
        ### -Pij / Pjj otherwise
        ### where P = Xt * X - lambda*I
        
        g = matrix.T.dot(matrix).toarray() 
        
        diagonal = np.diag_indices(g.shape[0])
        g[diagonal] += lambda_ ### => gives P
        
        p = np.linalg.inv(g)
        
        b = p / (-np.diag(p)) ### => gives Bij
        b[diagonal] = 0       ### and sets diagonal to 0
        
        self.b = b
        self.pred = matrix.dot(b)
        
    
    def predict(self, user_id, k):
        
        user_prediction = self.pred[user_id, :]
        item_ranked = np.argsort(-user_prediction)[:k]
        
        item_ranked = [self.item_dict[item] for item in item_ranked]
        
        return item_ranked
    
    
    def get_similarity(self, k=10):
        
        similarity = {}
        
        for idx, row in enumerate(self.b):
            
            sorted_index = np.argsort(-row)[:k]
            sorted_codes = [self.item_dict[item] for item in sorted_index]
            similarity.update({self.item_dict[idx]: sorted_codes})
            
        return similarity

In [69]:
### Main

In [71]:
dataset = Dataset(brand='missoni')
dataset.fit(metadata=['saleLine'])

In [72]:
config = {'model': ['lightfm', 'mlp'],
          'metadata': [True, False],
          'n_factors': [64, 128]}

is_verbose=True
epochs=20

In [73]:
import datetime

In [80]:
performance = None
id = 0
batch_size = int(dataset.dataset.shape[0] / 1000)
print(f'Batch Size: {batch_size}') 

for model in config['model']:
  for is_metadata in config['metadata']:
    for factors in config['n_factors']:

      print(f'Model: {model}, Metadata: {is_metadata}, Factors: {factors} \n')
      start = datetime.datetime.now()

      if model == 'lightfm':

        modello = LightFM(dataset=dataset,
                          n_factors=factors,
                          use_metadata=is_metadata,
                          use_cuda=True)
        
      else: 

        modello = MLP(dataset=dataset,
                          n_factors=factors,
                          use_metadata=is_metadata,
                          use_cuda=True)
        
      optimizer_model = torch.optim.Adam(modello.parameters(), 
                                    lr=1e-3,
                                    weight_decay=1e-6)
      
      modello.fit(optimizer=optimizer_model, 
                  batch_size=batch_size, 
                  epochs=epochs, 
                  split_train_test=True,
                  verbose=is_verbose)
      
      end = datetime.datetime.now()
      runtime = (end - start).seconds

      print(f'\n Runtime: {runtime} \n')

      id += 1
      
      single = pd.DataFrame({'id': id,
                            'model': model,
                            'use_metadata': is_metadata,
                            'factors': factors,
                            'last_train_AUC': modello.history()['train_auc'][epochs-1],
                            'last_test_AUC': modello.history()['test_auc'][epochs-1],
                            'train_AUC': [modello.history()['train_auc']],
                            'test_AUC': [modello.history()['test_auc']],
                            'runtime': [runtime]})
      
      performance = pd.concat([performance, single])

Batch Size: 726
Model: lightfm, Metadata: True, Factors: 64 

|== Splitting Train/Test ==|
Shape Train: 653454 
Shape Test: 72606
Epoch: 1
Epoch 1, 
== Loss: 0.4897219240665436 
== Train AUC: 0.82095 
== Test AUC: 0.8065
Epoch: 2
Epoch 2, 
== Loss: 0.2467082142829895 
== Train AUC: 0.844 
== Test AUC: 0.8291
Epoch: 3
Epoch 3, 
== Loss: 0.30169913172721863 
== Train AUC: 0.85465 
== Test AUC: 0.8421
Epoch: 4
Epoch 4, 
== Loss: 0.19369842112064362 
== Train AUC: 0.8699 
== Test AUC: 0.8508
Epoch: 5
Epoch 5, 
== Loss: 0.205636665225029 
== Train AUC: 0.8733 
== Test AUC: 0.8602
Epoch: 6
Epoch 6, 
== Loss: 0.151855006814003 
== Train AUC: 0.8869 
== Test AUC: 0.86915
Epoch: 7
Epoch 7, 
== Loss: 0.20444634556770325 
== Train AUC: 0.8932 
== Test AUC: 0.87515
Epoch: 8
Epoch 8, 
== Loss: 0.22585994005203247 
== Train AUC: 0.90345 
== Test AUC: 0.88135
Epoch: 9
Epoch 9, 
== Loss: 0.14001056551933289 
== Train AUC: 0.90635 
== Test AUC: 0.88395
Epoch: 10
Epoch 10, 
== Loss: 0.17071206867694855 

In [81]:
performance.sort_values('last_test_AUC', ascending=False)

,id,model,use_metadata,factors,last_train_AUC,last_test_AUC,train_AUC,test_AUC,runtime
0,3,lightfm,False,64,0.96225,0.91195,"[0.8318, 0.8811, 0.8979, 0.90605, 0.9163, 0.92...","[0.83285, 0.8638, 0.87405, 0.88985, 0.89425, 0...",95
0,4,lightfm,False,128,0.97480,0.91130,"[0.86535, 0.8979, 0.9109, 0.9215, 0.92905, 0.9...","[0.8477, 0.87415, 0.88465, 0.89475, 0.8984, 0....",128
0,2,lightfm,True,128,0.96295,0.91035,"[0.8235, 0.8393, 0.86525, 0.8796, 0.8949, 0.90...","[0.80845, 0.82815, 0.845, 0.862, 0.8722, 0.878...",150
0,1,lightfm,True,64,0.94785,0.90595,"[0.82095, 0.844, 0.85465, 0.8699, 0.8733, 0.88...","[0.8065, 0.8291, 0.8421, 0.8508, 0.8602, 0.869...",119
0,6,mlp,True,128,0.92370,0.89345,"[0.80285, 0.8467, 0.8609, 0.8746, 0.88295, 0.8...","[0.78675, 0.8362, 0.84805, 0.85635, 0.85995, 0...",158
0,5,mlp,True,64,0.91550,0.89040,"[0.7886, 0.82615, 0.8459, 0.85925, 0.865, 0.87...","[0.786, 0.81425, 0.8345, 0.84735, 0.849, 0.855...",126
0,8,mlp,False,128,0.92195,0.88475,"[0.7295, 0.7889, 0.8344, 0.8517, 0.86695, 0.88...","[0.7238, 0.783, 0.82135, 0.8374, 0.8461, 0.862...",142
0,7,mlp,False,64,0.91605,0.88145,"[0.7298, 0.7413, 0.81535, 0.84165, 0.8561, 0.8...","[0.73015, 0.7392, 0.80485, 0.829, 0.83455, 0.8...",107


In [ ]:
if is_verbose:
  import matplotlib.pyplot as plt
  plt.plot(range(epochs), modello.history()['train_auc'], label='train_meta_auc')
  plt.plot(range(epochs), modello.history()['test_auc'], label='test_meta_auc')